In [ ]:
import sys
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QWidget, QPushButton
import numpy as np
from queue import PriorityQueue
import time
import math

# Define the grid size and create a grid
grid_size = 15
grid = np.zeros((grid_size, grid_size))

# Define start location for the robot
start = (0, 0)
current_location = start

# Define locations for buildings, houses, delivery points, and vehicles
buildings = [(2, 3), (4, 5), (6, 8)]
houses = [(10, 12), (2, 7)]
vehicles = [(5, 10), (10, 6)]

# Generate delivery points randomly
num_delivery_points = 5
delivery_points = []
for _ in range(num_delivery_points):
    x = np.random.randint(grid_size)
    y = np.random.randint(grid_size)
    while (x, y) in buildings or (x, y) in houses or (x, y) in delivery_points:
        x = np.random.randint(grid_size)
        y = np.random.randint(grid_size)
    delivery_points.append((x, y))

# Mark buildings, houses, delivery points, and vehicles on the grid
for building in buildings:
    grid[building[0], building[1]] = 1

for house in houses:
    grid[house[0], house[1]] = 2

for point in delivery_points:
    grid[point[0], point[1]] = 3

for vehicle in vehicles:
    grid[vehicle[0], vehicle[1]] = 4

# Define Best First Search algorithm
def best_first_search(start, goal, grid):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current = frontier.get()

        if current == goal:
            break

        for next in get_neighbors(current, grid):
            new_cost = cost_so_far[current] + random_cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = heuristic(next, goal)
                frontier.put(next, priority)
                came_from[next] = current

    return came_from

# Define A* Search algorithm
def a_star_search(start, goal, grid):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while not frontier.empty():
        current = frontier.get()

        if current == goal:
            break

        for next in get_neighbors(current, grid):
            new_cost = cost_so_far[current] + random_cost(current, next)
            if next not in cost_so_far or new_cost < cost_so_far[next]:
                cost_so_far[next] = new_cost
                priority = new_cost + heuristic(next, goal)
                frontier.put(next, priority)
                came_from[next] = current

    return came_from
    
def get_neighbors(current, grid):
    neighbors = []
    x, y = current
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx != 0 or dy != 0:
                if 0 <= x + dx < len(grid) and 0 <= y + dy < len(grid[0]) and grid[x + dx][y + dy] != 1:
                    neighbors.append((x + dx, y + dy))
    return neighbors

def heuristic(current, goal):
    x1, y1 = current
    x2, y2 = goal
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)     

def random_cost(current, next):
    return np.random.randint(1, 21)  # Random cost between 1 and 20

# Define function to find path to delivery points sequentially
def find_path_to_delivery_points(start, delivery_points, grid):
    paths_to_points_best_first = {}
    paths_to_points_a_star = {}
    current_location = start
    
    for idx, point in enumerate(delivery_points):
        # Best First Search
        came_from_best_first = best_first_search(current_location, point, grid)
        path_best_first = []
        current_best_first = point
        while current_best_first != current_location:
            path_best_first.append(current_best_first)
            current_best_first = came_from_best_first[current_best_first]
        path_best_first.append(current_location)
        path_best_first.reverse()
        paths_to_points_best_first[idx + 1] = path_best_first
        current_location = point
        
        # A* Search
        came_from_a_star = a_star_search(current_location, point, grid)
        path_a_star = []
        current_a_star = point
        while current_a_star != current_location:
            path_a_star.append(current_a_star)
            current_a_star = came_from_a_star[current_a_star]
        path_a_star.append(current_location)
        path_a_star.reverse()
        paths_to_points_a_star[idx + 1] = path_a_star
        current_location = point
    
    return paths_to_points_best_first, paths_to_points_a_star

# Define the PyQt5 window class
class RobotMovementWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('Robot Movement Visualization')
        self.setGeometry(100, 100, 800, 600)

        # Create a Matplotlib figure and canvas
        self.figure, self.ax = plt.subplots()
        self.canvas = FigureCanvas(self.figure)

        # Create a button to start the robot movement animation
        self.start_button = QPushButton('Start Robot Movement', self)
        self.start_button.clicked.connect(self.animate_robot_movement)

        # Create a vertical layout for the widgets
        layout = QVBoxLayout()
        layout.addWidget(self.canvas)
        layout.addWidget(self.start_button)

        # Create a central widget to set the layout
        central_widget = QWidget()
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # Visualize the initial grid
        self.update_grid()
        
    def update_grid(self):
        self.ax.clear()
        self.ax.imshow(grid, cmap='viridis', interpolation='nearest')
        self.ax.set_title('City Area with Obstacles and Delivery Points')
        self.canvas.draw()

    def animate_robot_movement(self):
        # Call the find_path_to_delivery_points method to get paths
        paths_to_delivery_points = find_path_to_delivery_points(current_location, delivery_points, grid)
        paths_best_first, paths_a_star = paths_to_delivery_points

        for idx in paths_best_first:
         path_best_first = paths_best_first[idx]
         path_a_star = paths_a_star[idx]
         
        self.update_grid()

        
        # Choose the best path between Best First Search and A* Search

        if len(path_best_first) < len(path_a_star):
            best_path = path_best_first
            algorithm_used = 'Best First Search'
        else:
            best_path = path_a_star
            algorithm_used = 'A* Search'
        
        best_path=path_best_first
        # Update the grid visualization with the best path
        for i in range(len(best_path) - 1):
            x1, y1 = best_path[i]
            x2, y2 = best_path[i + 1]
            grid[x1, y1] = 5  # Mark the current cell as visited
            grid[x2, y2] = 4  # Mark the next cell as the robot's current location
            self.update_grid() 
            QApplication.processEvents() # Update the grid visualization
            time.sleep(0.5)  # Pause for visualization (adjust as needed)

        # Update the current location of the robot to the final delivery point
        self.current_location = best_path[-1]
        grid[self.current_location[0], self.current_location[1]] = 3  # Mark the final cell as the delivery point
        self.update_grid()  # Update the grid visualization
        QApplication.processEvents() 
        time.sleep(1)  # Pause for visualization (adjust as needed)

       
app = QApplication(sys.argv)
window = RobotMovementWindow()
window.show()
app.exec_()
app.quit()
# sys.exit(app.exec_())


